# LangChain Expression Language

In [ ]:
%pip install langchain faiss-cpu langchain_openai langchain-community --upgrade

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"

In [4]:
from langchain_core.runnables import RunnableMap, RunnablePassthrough, RunnableLambda
from langchain_core.prompts import format_document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from operator import itemgetter
from langchain_community.vectorstores.faiss import FAISS
from langchain_openai import OpenAIEmbeddings

---

## Adding In Conversational History:

In [5]:
vectorstore = FAISS.from_texts(
    [
        "James Phoenix works as a data engineering and LLM consultant at JustUnderstandingData",
        "James is 31 years old.",
    ],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

In [6]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [7]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [8]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [9]:
from typing import List, Union
from langchain.schema import HumanMessage, SystemMessage, AIMessage

def _format_chat_history(chat_history: List[Union[HumanMessage, SystemMessage, AIMessage]]) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, HumanMessage):
            buffer += "\nHuman: " + dialogue_turn.content
        elif isinstance(dialogue_turn, AIMessage):
            buffer += "\nAssistant: " + dialogue_turn.content
        elif isinstance(dialogue_turn, SystemMessage):
            buffer += "\nSystem: " + dialogue_turn.content
    return buffer

In [10]:
_inputs = RunnableMap(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: _format_chat_history(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI()

In [11]:
conversational_qa_chain.invoke(
    {
        "question": "where did James work?",
        "chat_history": [],
    }
)

AIMessage(content='James Phoenix was employed at JustUnderstandingData.')

---

## Adding Memory

In [12]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [13]:
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature=0)
    | StrOutputParser(),
}
# Now we retrieve the documents

# This is REALLY IMPORTANT as the chain above becomes StrOutputParser() so it will only have one key, which gets passed to the retriever!
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [14]:
inputs = {"question": "where did James Phoenix work?"}
result = final_chain.invoke(inputs)
print(result)

{'answer': AIMessage(content='James Phoenix was employed at JustUnderstandingData.'), 'docs': [Document(page_content='James Phoenix works as a data engineering and LLM consultant at JustUnderstandingData'), Document(page_content='James is 31 years old.')]}


In [15]:
# Note that the memory does not save automatically
# This will be improved in the future
# For now you need to save it yourself
memory.save_context(inputs, {"answer": result["answer"].content})

In [16]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='where did James Phoenix work?'),
  AIMessage(content='James Phoenix was employed at JustUnderstandingData.')]}

------------------------------------------